In [1]:
import pandas as pd
import numpy as np
data_path = './data/'

In [2]:
source_data = pd.read_csv(data_path+'total_data_without_aqfull.csv',index_col=0)
station_info = pd.read_csv(data_path+'air_quality_station_info.csv',index_col=0)
time_info = pd.read_csv(data_path+'holiday_info.csv')
stations = pd.read_csv(data_path+'air_quality_station_info.csv',index_col=0)

# Combine dataset and build feature

In [4]:
attr = ['stationId','station_type', 'weekday','hour','humidity','pressure','temperature','wind_speed'] +\
time_info.columns[-7:].tolist() +['PM2.5', 'PM10','O3']
source_data = source_data.merge(station_info,on='stationId',how='left')\
                         .drop(['year','month','day','weekday','hour'],axis=1)\
                         .merge(time_info,on='time',how='left')[attr]

build features without historical data

In [5]:
def get_onehot(narray, stationType_map, stationId_map):
    '''get_onehot(narray)
    Format four features: stationId(35), stationType(4), weekday(7), hour(24) into seventy bool-typed features.
    narray: air_quality[:,:4], namely all four features of training data 
    '''
    onehot_data = []
    for i in range(narray.shape[0]):
        onehot_row = np.zeros(70)
        # stationId
        onehot_row[stationId_map[narray[i,0]]] = 1
        # stationType
        onehot_row[35+stationType_map[narray[i,1]]] = 1
        # weekday
        onehot_row[39+int(narray[i,2])] = 1
        # hour
        onehot_row[46+int(narray[i,3])] = 1
        
        onehot_data.append(onehot_row)
    return np.array(onehot_data)


def build_features(train_X, stations, pollutant='PM2.5', length=24*3):
    '''
    train_data: source pre-train data.
    stations: information of each station, a dataframe.
    '''   
    stationId_map = {}
    for index, station in enumerate(sorted(stations.stationId.unique().tolist())):
        stationId_map[station] = index
    stationType_map = {}
    for index, stationtype in enumerate(stations.station_type.unique().tolist()):
        stationType_map[stationtype] = index
        
    onehot = get_onehot(train_X[:,:4], stationType_map, stationId_map)
    
    return np.hstack((onehot, train_X[:,4:]))

#### PM2.5

In [5]:
# 74 features
Traindata_PM25 = source_data.iloc[:,:-2].dropna()
PM25_X = build_features(Traindata_PM25.iloc[:,:-1].values,stations)
PM25_Y = Traindata_PM25.iloc[:,-1]

In [6]:
print(PM25_X.shape)
print(PM25_Y.shape)

(362916, 81)
(362916,)


In [9]:
from sklearn.model_selection import train_test_split

In [8]:
train_X, test_X, train_Y, test_Y = train_test_split(PM25_X, PM25_Y, test_size=0.2, random_state=10)

In [9]:
train_X.shape

(290332, 81)

In [10]:
from lightgbm import LGBMRegressor as LR

In [11]:
reg_pm25 = LR()
reg_pm25.fit(train_X,train_Y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [12]:
pred_Y = reg_pm25.predict(test_X)
validation_error = np.mean(np.abs(pred_Y - test_Y) / (pred_Y + test_Y) * 2)
print('Validation error of {0} is {1}'.format('PM2.5', str(validation_error)))

Validation error of PM2.5 is 0.6096708614695169


**PM10**

In [13]:
Traindata_PM10 = source_data.loc[:,source_data.columns[:-3].tolist()+['PM10']].dropna()
PM10_X = build_features(Traindata_PM10.iloc[:,:-1].values,stations)
PM10_Y = Traindata_PM10.iloc[:,-1]

In [14]:
train_X, test_X, train_Y, test_Y = train_test_split(PM10_X, PM10_Y, test_size=0.2, random_state=10)

In [15]:
reg_pm10 = LR()
reg_pm10.fit(train_X,train_Y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [16]:
train_X.shape

(244165, 81)

In [17]:
pred_Y = reg_pm10.predict(test_X)
validation_error = np.mean(np.abs(pred_Y - test_Y) / (pred_Y + test_Y) * 2)
print('Validation error of {0} is {1}'.format('PM10', str(validation_error)))

Validation error of PM10 is 0.4926954796747407


**O3**

In [7]:
Traindata_O3 = source_data.loc[:,source_data.columns[:-3].tolist()+['O3']].dropna()
O3_X = build_features(Traindata_O3.iloc[:,:-1].values,stations)
O3_Y = Traindata_O3.iloc[:,-1]

In [11]:
train_X, test_X, train_Y, test_Y = train_test_split(O3_X, O3_Y, test_size=0.2, random_state=10)

In [12]:
reg_o3 = LR()
reg_o3.fit(train_X,train_Y)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [13]:
train_X.shape

(291344, 81)

In [14]:
pred_Y = reg_o3.predict(test_X)
validation_error = np.mean(np.abs(pred_Y - test_Y) / (pred_Y + test_Y) * 2)
print('Validation error of {0} is {1}'.format('O3', str(validation_error)))

Validation error of O3 is 0.5622590821604236


# Prediction

Prepare testX

In [107]:
import datetime

In [108]:
time_info_with5 = pd.read_csv(data_path+'holiday_info_with5.csv')

In [109]:
time_info_with5['time'] = time_info_with5.apply(lambda row: datetime.datetime.strftime(datetime.datetime.strptime(row['time'], "%Y/%m/%d %H:%M"), "%Y-%m-%d %H:%M:%S"), axis=1)

In [110]:
final_weather = pd.read_csv(data_path + 'MSBD5002PROJECT_data/gridWeather_20180501-20180502.csv')
geo_info = pd.read_csv(data_path + './geo_info_new.csv',index_col=None)

In [111]:
final_weather = final_weather.rename(columns={"station_id":"nearest_gw"})
aq_gw_foreign = geo_info[["aq_name","nearest_gw"]]
final_weather = final_weather.join(aq_gw_foreign.set_index("nearest_gw"),on="nearest_gw")
final_weather = final_weather[final_weather['aq_name'].isna()==False]\
                    [['aq_name','time','humidity','pressure','temperature','wind_speed']]\
                    .rename(columns={"aq_name":"stationId"})

In [112]:
attr = ['stationId','station_type', 'weekday','hour','humidity','pressure','temperature','wind_speed'] + time_info_with5.columns[-7:].tolist() 
final_weather = final_weather.merge(station_info,on='stationId',how='left')\
                             .join(time_info_with5.set_index('time'), on='time')[attr]

In [122]:
final_test_X = final_weather.sort_values(by=['stationId','weekday','hour']).reset_index(drop=True)

In [123]:
final_test_X

,stationId,station_type,weekday,hour,humidity,pressure,temperature,wind_speed,is_weekend,is_holiday,is_working,is_holiday_first,is_holiday_last,is_working_first,is_working_last
0,aotizhongxin_aq,Urban Stations,1,0,36.0,1008.2952,19.0,14.11,0,1,0,0,1,0,0
1,aotizhongxin_aq,Urban Stations,1,1,31.0,1008.3878,21.0,15.90,0,1,0,0,1,0,0
2,aotizhongxin_aq,Urban Stations,1,2,27.0,1008.5077,21.0,17.56,0,1,0,0,1,0,0
3,aotizhongxin_aq,Urban Stations,1,3,24.0,1008.0310,21.0,18.39,0,1,0,0,1,0,0
4,aotizhongxin_aq,Urban Stations,1,4,20.0,1008.0282,22.0,18.29,0,1,0,0,1,0,0
5,aotizhongxin_aq,Urban Stations,1,5,21.0,1007.5121,23.0,17.89,0,1,0,0,1,0,0
6,aotizhongxin_aq,Urban Stations,1,6,22.0,1007.1123,22.0,16.09,0,1,0,0,1,0,0
7,aotizhongxin_aq,Urban Stations,1,7,22.0,1006.8159,21.0,12.67,0,1,0,0,1,0,0
8,aotizhongxin_aq,Urban Stations,1,8,24.0,1007.4049,21.0,8.88,0,1,0,0,1,0,0
9,aotizhongxin_aq,Urban Stations,1,9,25.0,1007.5851,19.0,6.02,0,1,0,0,1,0,0


In [126]:
final_test_X['PM2.5'] = reg_pm25.predict(build_features(final_test_X.values,stations))

In [ ]:
final_test_X['PM10'] = reg_pm25.predict(build_features(final_test_X.iloc[:,-1].values,stations))
final_test_X['O3'] = reg_pm25.predict(build_features(final_test_X.iloc[:,-2].values,stations))

In [ ]:
final_test_x.to_csv(data_path+'Submission.csv')